# A/B Testing and Power Analysis



We going to A/B test the new landing page of a webside. I got data from https://www.kaggle.com/datasets/zhangluyuan/ab-testing.

The test metric is Conversion Rate (CR):
$$
    CR = converted / (converted + notconverted)
$$

Hypothesis: Changing landing page will increase the CR of the web.


In [1]:
import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 

In [2]:
data = pd.read_csv("../data/ab_data.csv", parse_dates=["timestamp"])

In [3]:
data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [4]:
data.shape

(294478, 5)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   user_id       294478 non-null  int64         
 1   timestamp     294478 non-null  datetime64[ns]
 2   group         294478 non-null  object        
 3   landing_page  294478 non-null  object        
 4   converted     294478 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 11.2+ MB


In [6]:
data["group"].value_counts()

treatment    147276
control      147202
Name: group, dtype: int64

## Designing the A/B test in 3 steps
1. setup statistical hypothesis
2. calculate the appropriate sample size
3. determine the test durartion 

### 1. Setup Statistical Hypothesis
$$
    H_0 : \mu_c = \mu_t \\
    H_1 : \mu_c \neq \mu_t
$$

$\mu_c$ is sampling distribution mean of CR in control group  
$\mu_t$  is sampling distribution mean of CR in treatment group  



### 2. calculate the appropriate sample size
